In [1]:
from pathlib import Path
import nibabel as nib
import numpy as np
import h5py

In [2]:
lh_mtl_atlas = nib.load("./nsd-data/lh.MTL.nii.gz").get_fdata()
rh_mtl_atlas = nib.load("./nsd-data/rh.MTL.nii.gz").get_fdata()

In [3]:
for curr_region_idx in range(1, 10+1):

    lh_idx = np.where(lh_mtl_atlas==curr_region_idx)
    rh_idx = np.where(rh_mtl_atlas==curr_region_idx)

    all_sessions = []
    for sess in range(1, 40+1):
        file_path = Path(f"shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session{int(sess):02d}.hdf5")
        if file_path.is_file():
            print(f"FOUND {file_path}")
            curr_session = []
            with h5py.File(file_path, "r") as f:
                # print(f["betas"][0, :].shape)
                curr_session += [f["betas"][:, lh_idx[2][i], lh_idx[1][i], lh_idx[0][i]] for i in range(len(lh_idx[0]))]
                curr_session += [f["betas"][:, rh_idx[2][i], rh_idx[1][i], rh_idx[0][i]] for i in range(len(rh_idx[0]))]
                curr_session = np.array(curr_session)
                print(curr_session.shape)
            all_sessions.append(curr_session)

    all_sessions = np.concatenate(all_sessions, axis=1) # voxels, 750*sessions
    np.save(f"nsd-data/MTL-{curr_region_idx}_full.npy", all_sessions.T)

FOUND shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session01.hdf5
(345, 750)
FOUND shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session02.hdf5
(345, 750)
FOUND shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session03.hdf5
(345, 750)
FOUND shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session04.hdf5
(345, 750)
FOUND shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session05.hdf5
(345, 750)
FOUND shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session06.hdf5
(345, 750)
FOUND shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session07.hdf5
(345, 750)
FOUND shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session08.hdf5
(345, 750)
FOUND shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_

In [4]:
# make sure hdf5 is the same as nifti
betas_nifti = nib.load("shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session01.nii.gz")
betas_hdf5 = h5py.File("shared/NSD/nsddata_betas/ppdata/subj01/func1pt8mm/beta_fithrf_GLMdenoise_RR/betas_session01.hdf5", "r")

In [5]:
np.allclose(betas_nifti.slicer[...,0].get_fdata(), np.swapaxes(betas_hdf5["betas"][0, :, :, :], 0, 2))

True